In [1]:
import pandas as pd
import re
from datetime import datetime
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

In [65]:
try:
    conexion= psycopg2.connect(
        database= "Lab.14",
        user= "postgres",
        password= "admin",
        host= "localhost",
        port= "5432"
    )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("La contraseña es erronea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexion")
    else:
        print(f"Ocurrió el error {e}")

***TODOS LAS QUERIES QUE ESTÁN HECHAS DISTINTAS ES POR LA FALTA DE ESOS DATOS EN MI BASE DE DATOS***

## Ejercicio 1. Queries Generales

1.1. Calcula el promedio de temperatura.



In [5]:
cursor= conexion.cursor()
query_promedio= """
SELECT AVG(temperatura_°c)
FROM tiempo t;
"""
cursor.execute(query_promedio)   
cursor.fetchone()

(Decimal('21.2498387720882239'),)

1.2. Obtén los municipios en los cuales coincidan las medias de la sensación térmica y de la temperatura. 



Lo comento porque no tengo esa variable y para que no me de error

In [ ]:
# cursor=conexion.cursor()
# query_coincidencia="""
# SELECT  m."nombre_municipio", AVG(temperatura_°c),  AVG(sensacion_termica)
# FROM tiempo t
# INNER JOIN municipios m ON m."id_municipio" = t."id_municipio"
# GROUP BY id_municipio
# HAVING AVG(temperatura_°c) = AVG(sensacion_termica)

# """

1.3. Obtén el local más cercano de cada municipio



In [49]:
cursor = conexion.cursor()
query_distancia=""" 
SELECT  m.nombre_municipio, categorias, MIN(l.distancia) AS distancia1
FROM categorias c
INNER JOIN lugar l ON l.id_categoria = c.id_categorias
INNER JOIN municipios m ON l.id_municipio = m.id_municipio
GROUP BY id_categorias, m.id_municipio

"""
cursor.execute(query_distancia)
df_query3 = pd.DataFrame(cursor.fetchall(), columns=["Municipio","Categorias","Distancia"])
df_query3.head()
#AHORA VUELVO  ESTE

,Municipio,Categorias,Distancia
0,Robledo de Chavela,Park,977.0
1,Villanueva de la Cañada,Park,577.0
2,Collado Villalba,Park,210.0
3,Coslada,Park,70.0
4,Navacerrada,Park,95.0


1.4. Localiza los municipios que posean algún localizador a una distancia mayor de 2000 y que posean al menos 25 locales.



In [50]:
cursor = conexion.cursor()
query_distancia=""" 
SELECT * FROM (
            SELECT  m.nombre_municipio AS nombre_mun, count(c.id_categorias) AS numero_locales, sum(distancia) AS total_distancia
                FROM categorias c
                INNER JOIN lugar l ON l.id_categoria = c.id_categorias
                INNER JOIN municipios m ON l.id_municipio = m.id_municipio
                GROUP BY m.id_municipio)
WHERE (total_distancia > 2000 AND numero_locales > 25);
"""
cursor.execute(query_distancia)
df_query4 = pd.DataFrame(cursor.fetchall(), columns=["Municipio","Numero_locales","Distancia"])
df_query4.head()

,Municipio,Numero_locales,Distancia
0,Majadahonda,27,28639.0
1,Torrejón de Ardoz,30,36768.0
2,Madrid,122,138753.0
3,Alcobendas,53,62547.0
4,Alcalá de Henares,52,49584.0


1.5. Teniendo en cuenta que la lluvia se considera leve con entre 6 y 20 mm, moderada con una media de entre 21 y 40 mm, fuerte con media de entre 41 y 70 mm y muy fuerte entre 71 y 120 mm. Calcula cuántas luvias de cada tipo tenemos en cada uno de los días. Este ejercicio debes solucionarlo con la sentencia CASE de SQL (no la hemos visto en clase, por lo que tendrás que buscar la documentación). 



In [51]:
cursor = conexion.cursor()
query_lluvia="""
SELECT sum(precipitacion_mm), fecha,
CASE
    WHEN sum(precipitacion_mm) < 6 THEN 'sin lluvia'
    WHEN sum(precipitacion_mm) >= 6 AND sum(precipitacion_mm) <= 20 THEN 'leve'
    WHEN sum(precipitacion_mm) >= 21 AND sum(precipitacion_mm) <= 40 THEN 'media'
    WHEN sum(precipitacion_mm) >= 41 AND sum(precipitacion_mm) <= 70 THEN 'fuerte'
    WHEN sum(precipitacion_mm) >= 71 AND sum(precipitacion_mm) <= 120 THEN 'muy fuerte'
    ELSE 'torrencial'
    END AS lluvia_range
FROM tiempo t
GROUP BY fecha;
"""
cursor.execute(query_lluvia)
df_query5 = pd.DataFrame(cursor.fetchall(), columns=["Precipitaciones","Fecha","Calificacion"])
df_query5.head()

,Precipitaciones,Fecha,Calificacion
0,3.2,2024-08-29 15:00:00,sin lluvia
1,34.8,2024-08-29 19:00:00,media
2,46.3,2024-08-29 17:00:00,fuerte
3,14.1,2024-08-29 16:00:00,leve
4,0.0,2024-08-31 03:00:00,sin lluvia


## Ejercicio 2. Vistas

2.1. Crea una vista que muestre la información de los locales que tengan incluido el código postal en su dirección. 

Busca los municipios que no tengan lat y lon

In [56]:
#CREATE  VIEW VistaGastosCanciones5 AS
cursor= conexion.cursor()
query_muns_lat="""
CREATE VIEW VistaMunicipiosNoLatLong AS
SELECT nombre_municipio, latitud, longitud
FROM municipios m
WHERE latitud = NULL AND longitud = NULL;
"""
cursor.execute(query_muns_lat)

query_vista=""" 
SELECT * FROM VistaMunicipiosNoLatLong;
"""
cursor.execute(query_vista)
df_query6 = pd.DataFrame(cursor.fetchall(), columns=["Nombre municipio", "Latitud", "Longitud"])
df_query6.head()

,Nombre municipio,Latitud,Longitud


2.2. Crea una vista con los locales que tienen más de una categoría asociada.



In [58]:
cursor= conexion.cursor()
query_cat_repe="""
CREATE VIEW NumeroCatsDoble AS
SELECT id_categorias, count(categorias)
FROM categorias
GROUP BY id_categorias
HAVING count(categorias) != 1
"""

cursor.execute(query_cat_repe)

query_vista2=""" 
SELECT * FROM NumeroCatsDoble;
"""
cursor.execute(query_vista2)
df_query7 = pd.DataFrame(cursor.fetchall(), columns=["Local", "Categoria"])
df_query7.head()

,Local,Categoria


2.3. Crea una vista que muestre el municipio con la temperatura más alta de cada día



In [60]:
cursor= conexion.cursor()
query_temp="""
CREATE VIEW MunicipioTempAlta AS
SELECT nombre_municipio, MAX(temperatura_°c) AS temperatura_max
FROM tiempo t
INNER JOIN municipios m ON t.id_municipio = m.id_municipio
GROUP BY fecha, m.id_municipio
"""
cursor.execute(query_temp)

query_vista3=""" 
SELECT * FROM MunicipioTempAlta;
"""
cursor.execute(query_vista3)

df_query8 = pd.DataFrame(cursor.fetchall(), columns=["Municipio", "TemperaturaMAx"])
df_query8.head()

,Municipio,TemperaturaMAx
0,Villaviciosa de Odón,25
1,Collado Villalba,18
2,Madarcos,16
3,Sevilla la Nueva,19
4,Pozuelo del Rey,16


2.4. Crea una vista con los municipios en los que haya una probabilidad de precipitación mayor del 100% durante mínimo 7 horas.



In [ ]:
cursor= conexion.cursor()
query_humedad="""
CREATE VIEW MunicipioHumedo AS
SELECT nombre_municipio, 
FROM tiempo t
INNER JOIN municipios m ON m.id_municipio = t.id_municipio
WHERE humedad > 70 
"""
cursor.execute(query_humedad)

query_vista4=""" 
SELECT * FROM MunicipioHumedo;
"""
cursor.execute(query_vista4)

df_query9 = pd.DataFrame(cursor.fetchall(), columns=["Municipio", "TemperaturaMAx"])
df_query9.head()



#LUEGO VUELVO A ESTE

2.5. Obtén una lista con los parques de los municipios que tengan algún castillo.

In [63]:
cursor= conexion.cursor()
query_parques="""
CREATE VIEW ParqueCastillo AS
SELECT l.nombre FROM municipios m 
INNER JOIN lugar l 
		ON l.id_municipio = m.id_municipio 
INNER JOIN categorias c  
		ON c.id_categorias = l.id_categoria 
WHERE c.categorias LIKE 'Castle' AND c.categorias LIKE 'Park'
"""
cursor.execute(query_parques)
query_vista5=""" 
SELECT * FROM ParqueCastillo;
"""
cursor.execute(query_vista5)

df_query10 = pd.DataFrame(cursor.fetchall(), columns=["Nombre_parque"])
df_query10.head()

,Nombre_parque


## Ejercicio 3. Tablas Temporales

3.1. Crea una tabla temporal que muestre cuántos días han pasado desde que se obtuvo la información de la tabla AEMET.



**QUEDA PENDIENTE INTENTAR SACARLOS**

In [ ]:
#LUEGO VEO

3.2. Crea una tabla temporal que muestre los locales que tienen más de una categoría asociada e indica el conteo de las mismas



3.3. Crea una tabla temporal que muestre los tipos de cielo para los cuales la probabilidad de precipitación mínima de los promedios de cada día es 5.



3.4. Crea una tabla temporal que muestre el tipo de cielo más y menos repetido por municipio.

In [ ]:
#EL ROW_NUMBER() asigna un número de fila para cada registro
#Se agrupan los resultados por id_municipio
#fila_desc es el número de fila asignado a cada cielo(es 1 para el más repetido)(fila_asc parecido pero con el menos repetido)
#fila_desc:Es un número de fila que asignamos a cada tipo de cielo según su frecuencia de aparición,
# en orden descendente. El tipo de cielo más repetido (el que tiene el mayor total_cielo)
#recibe el número de fila 1, el segundo más repetido recibe el número 2, y así sucesivamente
#Aquí se selecciona el cielo que tiene fila_asc = 1(el cielo menos repetido)


In [66]:
cursor= conexion.cursor()
query_tipo="""
CREATE TEMPORARY TABLE MasMenosTipoCielo AS
WITH cielo_conteo AS (
    SELECT 
        t.id_municipio,
        c.cielo,
        COUNT(c.cielo) AS total_cielo,
        ROW_NUMBER() OVER (PARTITION BY t.id_municipio ORDER BY COUNT(c.cielo) DESC) AS fila_desc,       
        ROW_NUMBER() OVER (PARTITION BY t.id_municipio ORDER BY COUNT(c.cielo) ASC) AS fila_asc          
    FROM cielo c                                                                                         
    INNER JOIN tiempo t ON c.id_cielo = t.id_cielo
    GROUP BY t.id_municipio, c.cielo
)


SELECT 
    MAX(CASE WHEN fila_desc = 1 THEN cielo END) AS cielo_mas_repetido,                                   
    MAX(CASE WHEN fila_desc = 1 THEN total_cielo END) AS veces_mas_repetido,
    MAX(CASE WHEN fila_asc = 1 THEN cielo END) AS cielo_menos_repetido,                                  
    MAX(CASE WHEN fila_asc = 1 THEN total_cielo END) AS veces_menos_repetido,
    id_municipio
FROM cielo_conteo
GROUP BY id_municipio;
"""
cursor.execute(query_tipo)
query_vista8=""" 
SELECT * FROM MasMenosTipoCielo;
"""
cursor.execute(query_vista8)

df_query11 = pd.DataFrame(cursor.fetchall(), columns=["cielo mas repetido", "veces mas repetido", "cielo menos repetido", "veces menos repetido", "ID_municipio"])
df_query11.head()

,cielo mas repetido,veces mas repetido,cielo menos repetido,veces menos repetido,ID_municipio
0,Poco nuboso,11,Cubierto con tormenta y lluvia escasa,1,28001
1,Nubes altas,8,Muy nuboso con lluvia,1,28002
2,Poco nuboso,18,Muy nuboso con tormenta y lluvia escasa,1,28003
3,Nubes altas,9,Muy nuboso con lluvia escasa,1,28004
4,Poco nuboso,10,Nuboso con tormenta y lluvia escasa,1,28005





## Ejercicio 4. SUBQUERIES

4.1. Necesitamos comprobar si hay algún municipio en el cual no tenga ningún local registrado.



In [67]:
cursor= conexion.cursor()
query_munsin="""
SELECT * FROM (
			SELECT nombre_municipio, count(id_categorias) AS NumeroLocales
				FROM categorias c
				INNER JOIN lugar l ON c.id_categorias = l.id_categoria
				INNER JOIN municipios m ON l.id_municipio = m.id_municipio
				GROUP BY m.id_municipio)
WHERE NumeroLocales = 0;
"""
cursor.execute(query_munsin)

df_query12 = pd.DataFrame(cursor.fetchall(), columns=["Nombre Municipio", "Numero de locales"])
df_query12.head()

,Nombre Municipio,Numero de locales


4.2. Averigua si hay alguna fecha en la que el cielo se encuente "Muy nuboso con tormenta".



In [68]:
cursor= conexion.cursor()
query_malo="""
SELECT * FROM(
        SELECT t.fecha, c.cielo AS CieloTormentoso
            FROM cielo c
            INNER JOIN tiempo t ON c.id_cielo = t.id_cielo)
WHERE CieloTormentoso LIKE 'Muy nuboso con tormenta'
"""
cursor.execute(query_malo)

df_query13 = pd.DataFrame(cursor.fetchall(), columns=["Fecha", "Estado del cielo"])
df_query13.head()

,Fecha,Estado del cielo
0,2024-08-29 16:00:00,Muy nuboso con tormenta
1,2024-08-29 18:00:00,Muy nuboso con tormenta
2,2024-08-29 19:00:00,Muy nuboso con tormenta


4.3. Encuentra los días en los que los avisos sean diferentes a "Sin riesgo".



In [69]:
cursor= conexion.cursor()
query_tranquilo="""
SELECT * FROM(
        SELECT t.fecha, a.avisos AS AvisoPeligro
            FROM avisos a
            INNER JOIN tiempo t ON a.id_avisos = t.id_avisos)
WHERE AvisoPeligro LIKE 'Sin riesgo'
"""
cursor.execute(query_tranquilo)

df_query14 = pd.DataFrame(cursor.fetchall(), columns=["Fecha", "Estado del cielo"])
df_query14.head()

,Fecha,Estado del cielo
0,2024-08-30 00:00:00,Sin riesgo
1,2024-08-30 01:00:00,Sin riesgo
2,2024-08-30 02:00:00,Sin riesgo
3,2024-08-30 03:00:00,Sin riesgo
4,2024-08-30 04:00:00,Sin riesgo


4.4. Selecciona el municipio con mayor número de locales.



In [70]:
cursor= conexion.cursor()
query_mayornloc="""
SELECT * FROM (
			SELECT m.nombre_municipio AS Nombre_municipio, count(l.id_lugar) AS NumeroLoc
				FROM categorias c
				INNER JOIN lugar l ON c.id_categorias = l.id_categoria
				INNER JOIN municipios m ON l.id_municipio = m.id_municipio
				GROUP BY m.id_municipio)
ORDER BY NumeroLoc DESC
LIMIT 1;
"""
cursor.execute(query_mayornloc)

df_query15 = pd.DataFrame(cursor.fetchall(), columns=["Nombre Municipio", "Numero de locales"])
df_query15.head()

,Nombre Municipio,Numero de locales
0,Madrid,122


4.5. Obtén los municipios muya media de sensación térmica sea mayor que la media total.



In [71]:
#Lo hago de la temperatura porque no tengo sensación
cursor= conexion.cursor()
query_tempmed="""
SELECT m.nombre_municipio, AVG(t."temperatura_°c") AS Media_municipio 
FROM tiempo t
INNER JOIN municipios m ON t.id_municipio = m.id_municipio
GROUP BY m.id_municipio
HAVING AVG(t."temperatura_°c") > (SELECT Media_total FROM (
														SELECT AVG("temperatura_°c") AS Media_total
														    FROM tiempo));
"""
cursor.execute(query_tempmed)

df_query16 = pd.DataFrame(cursor.fetchall(), columns=["Nombre Municipio", "Temperatura media municipio"])
df_query16.head()

,Nombre Municipio,Temperatura media municipio
0,Titulcia,22.7727272727272727
1,Mejorada del Campo,23.0227272727272727
2,Casarrubuelos,22.7272727272727273
3,Valverde de Alcalá,21.3181818181818182
4,Rivas-Vaciamadrid,22.8863636363636364


4.6. Selecciona los municipios con más de dos fuentes.



In [72]:
cursor= conexion.cursor()
query_fuentes="""
SELECT nombre_municipio, count(c.id_categorias) AS NumeroFuentes
FROM categorias c
INNER JOIN lugar l ON c.id_categorias = l.id_categoria
INNER JOIN municipios m ON l.id_municipio = m.id_municipio
WHERE c.categorias LIKE 'Fountain'
GROUP BY m.id_municipio
HAVING COUNT(c.id_categorias) > 2;
"""

cursor.execute(query_fuentes)

df_query17 = pd.DataFrame(cursor.fetchall(), columns=["Nombre Municipio", "Numero fuentes"])
df_query17.head()

,Nombre Municipio,Numero fuentes
0,Madrid,3


4.7. Localiza la dirección de todos los estudios de cine que estén abiertod en el municipio de "Madrid".



In [73]:
#NO TENGO LA DIRECCION QUIZA PUEDO TRAER LA LATITUD EN SU LUGAR (SI ME DA TIEMPO AL ACABAR)

4.8. Encuentra la máxima temperatura para cada tipo de cielo.



No se me ocurre ninguna manera de hacerlo con subconsultas

In [74]:
cursor= conexion.cursor()
query_tempcielo="""
SELECT cielo, MAX(temperatura_°c)
FROM cielo c 
INNER JOIN tiempo t ON c.id_cielo = t.id_cielo 
GROUP BY c.cielo;
"""
cursor.execute(query_tempcielo)

df_query18 = pd.DataFrame(cursor.fetchall(), columns=["Estado del cielo", "Temperatura maxima"])
df_query18.head()

,Estado del cielo,Temperatura maxima
0,Intervalos nubosos con lluvia escasa,31
1,Nubes altas,31
2,Poco nuboso,32
3,Muy nuboso con tormenta,27
4,Muy nuboso con tormenta y lluvia escasa,28


4.9. Muestra el número de locales por categoría que muy probablemente se encuentren abiertos.



**TENGO PENDIENTE VER SI CONSIGO HACER QUE LA SEGUNDA FUNCIONE Y SINO ME QUEDE CON LA 1 (AUNQUE NO DA LO QUE SE PIDE)**

In [ ]:
cursor= conexion.cursor()
query_abiertos="""
SELECT count(id_categoria) AS Numero
FROM estados e 
INNER JOIN lugar l ON e.id_estados = l.id_estados 
WHERE estados LIKE 'VeryLikelyOpen'
"""
cursor.execute(query_tempcielo)

df_query18 = pd.DataFrame(cursor.fetchall(), columns=["Estado del cielo", "Temperatura maxima"])
df_query18.head()

In [ ]:
cursor= conexion.cursor()
query_abiertos="""
SELECT * FROM (
			SELECT count(c.id_categorias) AS Numero
				FROM estados e 
				INNER JOIN lugar l ON e.id_estados = l.id_estados 
				INNER JOIN categorias c ON l.id_categoria = c.id_categorias 
				GROUP BY c.categorias)
WHERE estados LIKE 'VeryLikelyOpen'
"""
cursor.execute(query_tempcielo)

df_query18 = pd.DataFrame(cursor.fetchall(), columns=["Estado del cielo", "Temperatura maxima"])
df_query18.head()

BONUS. 4.10. Encuentra los municipios que tengan más de 3 parques, los cuales se encuentren a una distancia menor de las coordenadas de su municipio correspondiente que la del Parque Pavia. Además, el cielo debe estar despejado a las 12.
